---
authors:
  - name: Mathis Derenne
    affiliation: M2 MIASHS - Université de Lyon
    github: mathisdrn
  - name: Djida Boukari
    affiliation: M2 MIASHS - Université de Lyon
  - name: Ines Nakhli
    affiliation: M2 MIASHS - Université de Lyon
date: 2026-02-01
---

# Introduction

### 1. Contexte et objectifs
Le projet SigBERT propose une nouvelle approche de modélisation du risque patient en transformant des séries temporelles d'embeddings de textes cliniques (OncoBERT) en signatures mathématiques. L'objectif de cette étude est de coupler cette représentation riche des données à des méthodes de prédiction conforme. Il s'agit de dépasser la simple estimation ponctuelle du risque fournie par les modèles de survie classiques pour offrir des intervalles de prédiction valides avec des garanties statistiques de couverture, un prérequis essentiel pour l'aide à la décision clinique.

### 2 Description des données
Les données sont issues des fichiers `df_study_L18_w6.csv` et `df_study_L36_w6.csv` disponibles dans le dossier `Atelier_SigBERT` du dépot GitHub [MINCHELLA-Paul/Master-MIASHS](https://github.com/MINCHELLA-Paul/Master-MIASHS/).

**Structure et Notation**

Soit un jeu de données $\mathcal{D} = \{(X_i, T_i, \delta_i)\}_{i=1}^n$ où chaque patient $\ni$ est défini par :
- **Covariables ($X_i$) :** Les coefficients issus des signatures de chemins. Ces vecteurs de haute dimension capturent les interactions non-linéaires et l'ordre séquentiel des événements cliniques.
- **Temps observé ($T_i$) :** $T_i = \min(T^*_i, C_i)$, où $T^*_i$ est le temps réel jusqu'au décès et $C_i$ le temps de censure.
- **Indicateur de censure ($\delta_i$) :** $\delta_i = \mathbb{1}(T^*_i \leq C_i)$.
  - $\delta_i = 1$ : Décès observé (événement).
  - $\delta_i = 0$ : Censure (perdu de vue ou fin d'étude).

**Prétraitement**
- Données manquantes : les jeux de données n'ont pas de données manquantes, aucune imputation n'est nécessaire.
- Standardisation : Centrage et réduction ($Z$-score) appliqués aux covariables $X$.
- Réduction de dimension : Application de l'ACP pour réduire la dimensionnalité des signatures tout en conservant 90% de la variance.
- Partitionnement : Division en ensemble d'entraînement, de test et de calibration pour la prédiction conforme.

### 3. Approche méthodologique

On propose d'entraîner une variété de modèle de survie adapté aux données censurées et de haute dimension. On conformalise ensuite les prédictions réalisées par le meilleur modèle selon les critères de performance classiques (C-index, Brier score).

Voici les modèles implémentés et comparés : 
- Modèle de Cox (L1, L2, ElasticNet, et en testant différent coefficients de régularisation)
- Random Survival Forest
- Gradient Boosting Survival Analysis
- Survival SVM
- XGBoost AFT

### 4. Prédiction conforme

La prédiction conforme standard (Split Conformal Prediction) repose sur l'hypothèse que les données sont échangeables. Cependant, l'analyse de survie introduit une complexité majeure : la **censure à droite**.

#### 4.1 Le défi de la censure

Dans notre jeu de données, pour un patient censuré ($\delta_i = 0$), le temps réel de survie $T^*_i$ est inconnu (nous savons seulement que $T^*_i > C_i$). Calculer un score de non-conformité classique (ex: résidu absolu) est donc impossible pour ces individus.
Exclure simplement les patients censurés de l'ensemble de calibration introduirait un **biais de sélection** massif : les patients dont l'événement est observé ont tendance à avoir des survies plus courtes que ceux qui sont censurés.

#### 4.2 Méthodologie : Weighted Split Conformal Prediction (WSCP)

Pour pallier ce problème, nous avons implémenté une approche de **Weighted Split Conformal Prediction** basée sur la pondération par l'inverse de la probabilité de censure (IPCW - *Inverse Probability of Censoring Weighting*). Cette méthode permet d'utiliser les données observées tout en corrigeant leur représentativité statistique par rapport à la population globale.

Nous appliquons cette procédure au modèle **XGBoost AFT** (Accelerated Failure Time), sélectionné à l'étape précédente, car il modélise directement le logarithme du temps de survie (($\log(T) \sim \mathcal{N}(f(X), \sigma)$), facilitant le calcul des résidus.

#### 4.3 Algorithme implémenté

La procédure se déroule en six étapes clés :

1. **Partitionnement (Splitting) :** Le jeu d'entraînement initial est divisé en un ensemble d'*Entraînement Propre* (80%) pour ajuster le modèle XGBoost, et un ensemble de *Calibration* (20%) $\mathcal{D}_{cal}$ réservé au calcul des seuils de conformité.
2. **Estimation de la censure :** Nous estimons la distribution de la censure $G(t) = P(C > t)$ sur l'ensemble de calibration à l'aide de l'estimateur de **Kaplan-Meier**, en inversant l'indicateur d'événement (considérant la censure comme l'événement d'intérêt).
3. **Calcul des scores de non-conformité :** Pour chaque patient $i$ **non-censuré** de l'ensemble de calibration, nous calculons le résidu absolu standardisé sur l'échelle logarithmique : $$s_i = |\log(T_i) - \log(\hat{T}_i)|$$
où $T_i$ est le temps observé et $\hat{T}_i$ la prédiction du modèle.

4. **Pondération (IPCW) :** Afin de compenser la sous-représentation des temps longs due à la censure, chaque score $s_i$ se voit attribuer un poids $w_i$ :$$w_i = \frac{1}{\hat{G}(T_i)}$$

Les patients ayant subi un événement à un temps où la censure est forte (i.e., $\hat{G}(t)$ est faible) reçoivent un poids plus important.

5. **Calcul du quantile pondéré :** Nous déterminons le quantile pondéré $1-\alpha$ de la distribution des scores $s_i$, noté $\hat{q}$. Ce seuil garantit que la somme des poids des scores inférieurs à $\hat{q}$ représente une fraction $1-\alpha$ de la masse totale des poids.
6. **Construction des intervalles :** Pour un nouveau patient $X_{n+1}$ du jeu de test, l'intervalle de prédiction à $1-\alpha$ est donné par :$$\hat{C}(X_{n+1}) = \left[ \exp(\hat{f}(X_{n+1}) - \hat{q}) \;, \; \exp(\hat{f}(X_{n+1}) + \hat{q}) \right]$$

L'utilisation de l'exponentielle permet de projeter l'intervalle calculé sur l'échelle log-normale vers l'échelle temporelle réelle (en jours).

### 5. Critères d'évaluation

La performance sera jugée sur deux axes :
1. **Discrimination et calibration (modèle de base) :** C-index, AUC dépendante du temps, IBS et Score de Brier à horizon de survie médian.
2. **Qualité des intervalles (prédiction conforme) :**
   - Couverture Marginale : Doit être statistiquement proche de $1-\alpha$.
   - Taille moyenne des ensembles de prédictions : plus les ensembles sont petits, plus ils sont informatifs.